In [605]:
import statsmodels.api as sm 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [629]:
df = pd.read_csv('../data/details_stats.csv')
ballon_dor  = pd.read_csv('../data/BallonDOr_combined.csv')

In [653]:
ballon_dor['Year'] = ballon_dor['Year'].apply(lambda x: int(str(x)[-2:]))
ballon_dor = ballon_dor.query('Year >= 8 & Year <=23')
# ballon_dor['Share'] = np.log(ballon_dor['Share'])

df['Season'] = df['Season'].apply(lambda x: int(str(x)[-2:]))
df = df.query('Season >= 8 & Season <=23')

In [654]:
bdo23 = ballon_dor.copy()
# bdo23 = ballon_dor.query('Year == 23')
bdo23 = bdo23[['Player', 'Share', 'Year']]

stats23 = df.copy()
stats23 = stats23.rename(columns={'Season' : 'Year'})
stats23 = stats23.replace('-', 0)

stats23 = pd.merge(stats23, bdo23, on=['Player', 'Year'], how='inner')

# stats23 = pd.get_dummies(stats23, columns=['Competition'])
stats23 = stats23.drop(columns=[ 'Club', 'Competition'])

stats23 = stats23.replace(np.nan, 0)
stats23 = stats23.query('Share != -inf')

stats23 = stats23.dropna(subset=stats23.select_dtypes(include=[float]).columns)
stats23 = stats23.set_index('Player')
stats23 = stats23.astype(float)
aggs = {i: 'sum' for i in stats23.columns if i != 'Share'}
aggs['Share'] = 'mean'
stats23 = stats23.groupby(['Player', 'Year']).agg(aggs)

In [658]:
stats23 = stats23.drop(columns=['Year'])

In [665]:
stats23

const  1 - 15  16 - 30  31 - 45  45+  46 - 60  \
Player             Year                                                  
Alexis Sánchez     15.0    1.0     2.0      4.0      5.0  2.0      3.0   
Andrea Pirlo       12.0    1.0     0.0      2.0      0.0  0.0      0.0   
                   13.0    1.0     3.0      0.0      0.0  0.0      2.0   
Andrei Arshavin    8.0     1.0     2.0      3.0      2.0  0.0      0.0   
                   9.0     1.0     0.0      0.0      1.0  0.0      0.0   
...                        ...     ...      ...      ...  ...      ...   
Zlatan Ibrahimović 15.0    1.0     6.0      4.0      3.0  0.0      4.0   
                   16.0    1.0     4.0      2.0      9.0  0.0     12.0   
Ángel Di María     14.0    1.0     1.0      3.0      0.0  0.0      2.0   
Éric Abidal        11.0    1.0     0.0      0.0      0.0  0.0      0.0   
İlkay Gündoğan     23.0    1.0     2.0      3.0      2.0  0.0      4.0   

                         61 - 75  76 - 90  90+  Extra time  Total   Share  
Player             Year                                                    
Alexis Sánchez     15.0      4.0      2.0  2.0         1.0   25.0  0.0114  
Andrea Pirlo       12.0      1.0      0.0  0.0         0.0    3.0  0.0266  
                   13.0      0.0      0.0  0.0         0.0    5.0  0.0111  
Andrei Arshavin    8.0       2.0      2.0  1.0         0.0   12.0  0.0444  
                   9.0       3.0      2.0  0.0         0.0    6.0  0.0035  
...                          ...      ...  ...         ...    ...     ...  
Zlatan Ibrahimović 15.0      5.0      7.0  1.0         0.0   30.0  0.0105  
                   16.0     11.0     11.0  1.0         0.0   50.0  0.0032  
Ángel Di María     14.0      4.0      1.0  0.0         0.0   11.0  0.0129  
Éric Abidal        11.0      1.0      0.0  0.0         0.0    1.0  0.0036  
İlkay Gündoğan     23.0      0.0      0.0  0.0         0.0   11.0  0.0054  

[359 rows x 12 columns]

In [678]:
glm1 =  sm.GLM(endog=stats23['Share'], exog=stats23.drop(columns=['Share']), family=sm.families.Gaussian())
stats23 = sm.add_constant(stats23)
results = glm1.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                  359
Model:                            GLM   Df Residuals:                      349
Model Family:                Gaussian   Df Model:                            9
Link Function:               Identity   Scale:                       0.0046316
Method:                          IRLS   Log-Likelihood:                 460.79
Date:                Fri, 15 Dec 2023   Deviance:                       1.6134
Time:                        13:56:09   Pearson chi2:                     1.61
No. Iterations:                     4   Pseudo R-squ. (CS):             0.3759
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0127      0.006     -2.086      0.037      -0.025      -0.001
1 - 15        -0.0039      0.003     -1.522      0.128      -0.009       0.001
16 - 30        0.0048      0.002      2.006      0.045       0.000       0.010
31 - 45       -0.0040      0.002     -1.698      0.090      -0.009       0.001
45+           -0.0144      0.006     -2.264      0.024      -0.027      -0.002
46 - 60       -0.0050      0.002     -2.194      0.028      -0.010      -0.001
61 - 75       -0.0066      0.002     -3.054      0.002      -0.011      -0.002
76 - 90        0.0020      0.002      0.837      0.403      -0.003       0.007
90+            0.0060      0.005      1.339      0.181      -0.003       0.015
Extra time     0.0255      0.010      2.592      0.010       0.006       0.045
Total          0.0045      0.001      3.451      0.001       0.002       0.007
==============================================================================
"""

In [685]:
stats23_2 = stats23.copy()
stats23_2 = stats23_2.drop(columns=['1 - 15', '31 - 45', '76 - 90', '90+'])

In [686]:
glm2 =  sm.GLM(endog=stats23_2['Share'], exog=stats23_2.drop(columns=['Share']), family=sm.families.Gaussian())
results = glm2.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                  359
Model:                            GLM   Df Residuals:                      352
Model Family:                Gaussian   Df Model:                            6
Link Function:               Identity   Scale:                       0.0046930
Method:                          IRLS   Log-Likelihood:                 456.56
Date:                Fri, 15 Dec 2023   Deviance:                       1.6519
Time:                        13:58:49   Pearson chi2:                     1.65
No. Iterations:                     3   Pseudo R-squ. (CS):             0.3573
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0138      0.006     -2.267      0.023      -0.026      -0.002
16 - 30        0.0055      0.002      2.309      0.021       0.001       0.010
45+           -0.0145      0.007     -2.083      0.037      -0.028      -0.001
46 - 60       -0.0037      0.002     -1.613      0.107      -0.008       0.001
61 - 75       -0.0054      0.002     -2.463      0.014      -0.010      -0.001
Extra time     0.0308      0.011      2.842      0.004       0.010       0.052
Total          0.0035      0.001      4.043      0.000       0.002       0.005
==============================================================================
"""

In [687]:
stats23_3 = stats23_2.copy()
stats23_3 = stats23_3.drop(columns=['46 - 60'])

In [688]:
glm3 =  sm.GLM(endog=stats23_3['Share'], exog=stats23_3.drop(columns=['Share']), family=sm.families.Gaussian())
results = glm3.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                  359
Model:                            GLM   Df Residuals:                      353
Model Family:                Gaussian   Df Model:                            5
Link Function:               Identity   Scale:                       0.0047143
Method:                          IRLS   Log-Likelihood:                 455.24
Date:                Fri, 15 Dec 2023   Deviance:                       1.6641
Time:                        13:59:45   Pearson chi2:                     1.66
No. Iterations:                     3   Pseudo R-squ. (CS):             0.3513
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0141      0.006     -2.316      0.021      -0.026      -0.002
16 - 30        0.0066      0.002      2.880      0.004       0.002       0.011
45+           -0.0143      0.007     -2.040      0.041      -0.028      -0.001
61 - 75       -0.0046      0.002     -2.160      0.031      -0.009      -0.000
Extra time     0.0325      0.011      3.013      0.003       0.011       0.054
Total          0.0026      0.001      3.938      0.000       0.001       0.004
==============================================================================
"""